In [ ]:
import datetime

ipfile = open("input.txt", "r")

In [ ]:
input = ipfile.readlines()
input

['Start state\n',
 '1 2 3\n',
 'B 4 6\n',
 '7 5 8\n',
 '\n',
 'Goal state\n',
 '1 2 3\n',
 '4 5 6\n',
 '7 8 B']

In [ ]:
#obtain start state from input
r1 = input[1]
start = r1.split(' ')
start[2] = start[2].split('\n')[0]
 
r2 = input[2]
start2 = r2.split(' ')
start = start + start2
start[5] = start[5].split('\n')[0]
 
r3 = input[3]
start3 = r3.split(' ')
start = start + start3
start[8] = start[8].split('\n')[0]
 
for n, i in enumerate(start):
  if i == 'B':
    start[n] = 0
 
for i in range(0,9):
  start[i] = int(start[i])

In [ ]:
#obtain goal state from input
g1 = input[6]
goal = g1.split(' ')
goal[2] = goal[2].split('\n')[0]
 
g2 = input[7]
goal2 = g2.split(' ')
goal = goal + goal2
goal[5] = goal[5].split('\n')[0]
 
g3 = input[8]
goal3 = g3.split(' ')
goal = goal + goal3
goal[8] = goal[8].split('\n')[0]
 
for n, i in enumerate(goal):
  if i == 'B':
    goal[n] = 0
 
for i in range(0,9):
  goal[i] = int(goal[i])

In [ ]:
print(start)
print(goal)

[1, 2, 3, 0, 4, 6, 7, 5, 8]
[1, 2, 3, 4, 5, 6, 7, 8, 0]


In [ ]:
print("Start state: \n")
i=0
while i<9:
    if(i%3 == 0):
      print("\n")
    print(str(start[i]) + '   ' + str(start[i+1]) + '   ' + str(start[i+2]))
    i=i+3

Start state: 



1   2   3


0   4   6


7   5   8


In [ ]:
print("Goal state: \n")
i=0
while i<9:
    if(i%3 == 0):
      print("\n")
    print(str(goal[i]) + '   ' + str(goal[i+1]) + '   ' + str(goal[i+2]))
    i=i+3

Goal state: 



1   2   3


4   5   6


7   8   0


In [ ]:
def getHeuristicMisplacedTiles(start, goal):
  '''Heuristic value uing number of misplaced tiles'''
  h = 0;
  for i in range(0,9):
    if start[i] != goal[i] and start[i]!=0:
      h=h+1
  return h

In [ ]:
def getHeuristicManhattanDistance(start, goal):
  '''Heuristic value uing manhattan distance'''
  return sum(abs(b%3 - g%3) + abs(b//3 - g//3)
        for b, g in ((start.index(i), goal.index(i)) for i in range(1, 9)))

In [ ]:
L = [] #L is the list of possible states to be explored
Lseen = [] #Lseen is the list of states which have already been explored
cost = []
level = 0
optimalCost = 0

In [ ]:
L.append(start) #initially only start state is in L

In [ ]:
level = 0
h = getHeuristicMisplacedTiles(start, goal)
cost.append(h)

In [ ]:
statesExplored = 0
statesOnOptimalPath = 0

In [ ]:
def expandNode(state, heuristicType, statesExplored):
  '''Generate all possible neighbours of a state'''
  zerothIndex = state.index(0)
  newStates = [] #[[1,2,3],[2,3,5]]
  finalStates = []
  newState = []  
  if (zerothIndex + 1) % 3 != 0: #move blank to right
    newState = state[:]
    temp = state[zerothIndex + 1]
    newState[zerothIndex + 1] = 0
    newState[zerothIndex] = temp
    newStates.append(newState)
    newState = []
  
  if (zerothIndex % 3) != 0: #move blank to left
    newState = state[:]
    temp = state[zerothIndex - 1]
    newState[zerothIndex - 1] = 0
    newState[zerothIndex] = temp
    newStates.append(newState)
    newState = []

  if zerothIndex not in [0,1,2]: #move blank up
    newState = state[:]
    temp = state[zerothIndex - 3]
    newState[zerothIndex - 3] = 0
    newState[zerothIndex] = temp
    newStates.append(newState)
    newState = []

  if zerothIndex not in [6,7,8]: #move blank down
    newState = state[:]
    temp = newState[zerothIndex + 3]
    newState[zerothIndex + 3] = 0
    newState[zerothIndex] = temp
    newStates.append(newState)
    newState = []

  for newState in newStates:
    statesExplored = statesExplored + 1
    finalStates.append(newState)
    
  return (statesExplored,finalStates)

In [ ]:
def getInversionCount(state):
    '''This is used to check if a given state is solveable or not'''
    inversionCount = 0
    for i in range(0,9):
      for j in range(i+1,9):
        if (state[i]>state[j] and state[i]!=0 and state[j]!= 0):
          inversionCount = inversionCount + 1
    print("inversionCount {}".format(inversionCount))
    return inversionCount

In [ ]:
def sortAndPlace(states,L,heuristicType):
  '''Sort the list of states as per their heuristic values'''
  costAndState = {}
  count = 0

  for state in states:
    if heuristicType == 1:
      costAndState[count] = getHeuristicMisplacedTiles(state, goal)
    else:
      costAndState[count] = getHeuristicManhattanDistance(state, goal)
    count = count + 1

  dict1 = dict(sorted(costAndState.items(), key=lambda item: item[1]))
  newList = []
  for key in list(dict1.keys()):
    newList.append(states[key])

  L = newList + L #newList is the sorted list of states as per heuristic value. It is prepended to L
  return L


In [ ]:
iteration = 0
inv = getInversionCount(start)
if 1 == 0: #inv % 2 != 0:
  print("This puzzle is not solveable")
else:
  print("Hill climbing search using Number of misplaced tiles as heuristic\n")
  stateToExpand = L[0]
  totalCost = 0
  statesInOptimalPath = 0
  t1 = datetime.datetime.now()
  prevState = start
  #Keep on iterating till the first value of L is not goal state OR if a local maxima or shoulder is acheieved 
  while(L[0] != goal and (getHeuristicMisplacedTiles(L[0],goal) < getHeuristicMisplacedTiles(prevState,goal) or iteration == 0)):
    iteration = iteration + 1
    stateToExpand = L[0]
    totalCost = totalCost + getHeuristicMisplacedTiles(stateToExpand,goal)
    statesInOptimalPath = statesInOptimalPath +1
    print("****************Iteration number {0}****************".format(iteration))
    i = 0
    while i<9:
      if(i%3 == 0):
        print("\n")
      print(str(stateToExpand[i]) + '   ' + str(stateToExpand[i+1]) + '   ' + str(stateToExpand[i+2]))
      i=i+3
    print('\n')
    (statesExplored,childStates) = expandNode(stateToExpand,1, statesExplored)#expand current state

    for state in childStates:
      if state in Lseen or state in L:
        statesExplored = statesExplored - 1
        childStates.remove(state) #remove states which have already been processed

    L = sortAndPlace(childStates,L,1)
    L.remove(stateToExpand)
    Lseen.append(stateToExpand)
    prevState = stateToExpand
    if(len(L) == 0):
      print("This puzzle is not solveable using Hill climbing") #If no more states to process then it is not solveable
      break

  if L[0] == goal:
    #print out the goal state as well
    stateToExpand = L[0]
    print("****************Iteration number {0}****************".format(iteration + 1))
    i = 0
    while i<9:
      if(i%3 == 0):
        print("\n")
      print(str(stateToExpand[i]) + '   ' + str(stateToExpand[i+1]) + '   ' + str(stateToExpand[i+2]))
      i=i+3

    t2 = datetime.datetime.now()
    t3 = t2 - t1
    print("Total number of states explored: {}".format(statesExplored))
    print("Total number of states in optimal path: {}".format(statesInOptimalPath + 1))
    print("Optimal path cost: {}".format(totalCost))
    print("Time taken for execution: {}".format(t3))


  else:
    print('Not solveable by hill climbing')


inversionCount 2
Hill climbing search using Number of misplaced tiles as heuristic

****************Iteration number 1****************


1   2   3


0   4   6


7   5   8


****************Iteration number 2****************


1   2   3


4   0   6


7   5   8


****************Iteration number 3****************


1   2   3


4   5   6


7   0   8


****************Iteration number 4****************


1   2   3


4   5   6


7   8   0
Total number of states explored: 8
Total number of states in optimal path: 4
Optimal path cost: 6
Time taken for execution: 0:00:00.006477


In [ ]:
L = []
Lseen = []
cost = []
level = 0
optimalCost = 0

In [ ]:
L.append(start)

In [ ]:
level = 0
h = getHeuristicManhattanDistance(start, goal)
cost.append(h)

In [ ]:
statesExplored = 0
statesOnOptimalPath = 0

In [ ]:
#This is exactly same as above only the heuristic type is changed
iteration = 0
inv = getInversionCount(start)
if 1 == 0: #inv % 2 != 0:
  print("This puzzle is not solveable")
else:
  print("Hill climbing search using Number of Manhattan tiles as heuristic\n")
  stateToExpand = L[0]
  totalCost = 0
  statesInOptimalPath = 0
  t1 = datetime.datetime.now()
  prevState = start
  while(L[0] != goal and (getHeuristicManhattanDistance(L[0],goal) < getHeuristicManhattanDistance(prevState,goal) or iteration == 0)):
    iteration = iteration + 1
    stateToExpand = L[0]
    totalCost = totalCost + getHeuristicManhattanDistance(stateToExpand,goal)
    statesInOptimalPath = statesInOptimalPath +1
    print("****************Iteration number {0}****************".format(iteration))
    i = 0
    while i<9:
      if(i%3 == 0):
        print("\n")
      print(str(stateToExpand[i]) + '   ' + str(stateToExpand[i+1]) + '   ' + str(stateToExpand[i+2]))
      i=i+3
    print('\n')
    (statesExplored,childStates) = expandNode(stateToExpand,1, statesExplored)#expand least cost node
    for state in childStates:
      if state in Lseen or state in L:
        statesExplored = statesExplored - 1
        childStates.remove(state)

    L = sortAndPlace(childStates,L,2)
    L.remove(stateToExpand)
    Lseen.append(stateToExpand)
    prevState = stateToExpand
    if(len(L) == 0):
      print("This puzzle is not solveable using Hill climbing")
      break

  if L[0] == goal:
    stateToExpand = L[0]
    print("****************Iteration number {0}****************".format(iteration + 1))
    i = 0
    while i<9:
      if(i%3 == 0):
        print("\n")
      print(str(stateToExpand[i]) + '   ' + str(stateToExpand[i+1]) + '   ' + str(stateToExpand[i+2]))
      i=i+3

    t2 = datetime.datetime.now()
    t3 = t2 - t1
    print("Total number of states explored: {}".format(statesExplored))
    print("Total number of states in optimal path: {}".format(statesInOptimalPath + 1))
    print("Optimal path cost: {}".format(totalCost))
    print("Time taken for execution: {}".format(t3))


  else:
    print('Not solveable by hill climbing')


inversionCount 2
Hill climbing search using Number of Manhattan tiles as heuristic

****************Iteration number 1****************


1   2   3


0   4   6


7   5   8


****************Iteration number 2****************


1   2   3


4   0   6


7   5   8


****************Iteration number 3****************


1   2   3


4   5   6


7   0   8


****************Iteration number 4****************


1   2   3


4   5   6


7   8   0
Total number of states explored: 8
Total number of states in optimal path: 4
Optimal path cost: 6
Time taken for execution: 0:00:00.013573
